In [1]:
import re
import os
import jieba
import random
import numpy as np
import pandas as pd
import networkx as nx

from pyltp import Postagger
from pyltp import Segmentor
from pyltp import NamedEntityRecognizer

from functools import partial
from collections import Counter
from scipy.spatial.distance import cosine

from gensim.test.utils import datapath
from gensim.corpora.dictionary import Dictionary
from gensim import corpora, models, similarities

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Summary_Model

In [11]:
class Summary_Model:
    def __init__(self):
        self.LTP_DATA_DIR = r'D:\ltp\ltp_data_v3.4.0'
#         self.cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
        self.pos_model_path = os.path.join(self.LTP_DATA_DIR, 'pos.model')  # 词性标注模型路径，模型名称为`pos.model`
        self.ner_model_path = os.path.join(self.LTP_DATA_DIR, 'ner.model')  # 命名实体识别模型路径，模型名称为`ner.model`
        self.embeddings_index_filename = r'D:\Assignment\datasource-master\sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5'
        self.stopwords_filename = "D:\Assignment\datasource-master\chinese_stopwords.txt"
        self.lda_model_filename = 'D:\\Assignment\\Project_03\\lda_news_model'
        self.lda_num = 100
        self.embedding_index = {}
        self.frequence = {}
        self.alpah = 1e-4
        self.knn_window = 1
        self.keywords_window = 3
        self.keywords_num = 5
        self.long_constraint=200
        self.title_weight = 0.5
        
    ################################################Process Data####################################################
    def token(self, string):
        return ''.join(re.findall(r'[\d|\w]+', string))
    
    def cut_sentence(self, para):
        para = re.sub('([；。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
        para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
        para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
        para = re.sub('([；。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    #     para = re.sub('([，,])', r'\1\n', para)
        # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
        para = para.rstrip()  # 段尾如果有多余的\n就去掉它
        # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
        return para.split("\n")
    
    def get_sentence_content_with_punctuation(self, news):
        news = re.sub(r'\n', '', news)
        if news != '' and news != ' ':
            result = self.cut_sentence(news)
        return result

    def get_sentence_content(self, news_sentence_with_punctuation):
        result = []

        for sentence in news_sentence_with_punctuation:
            temp = self.token(sentence)
            result.append(temp)
        return result

    def get_word_content(self, news_sentence):
        result = []
    #     segmentor = Segmentor()  # 初始化实例
    #     segmentor.load(cws_model_path)  # 加载模型

        for sentence in news_sentence:
            temp_words = list(jieba.cut(sentence))
    #         temp_words = list(segmentor.segment(sentence))  # 使用哈工大pyltp分词进行切词

            result.append(temp_words)

    #     segmentor.release()  # 释放模型
        return result

    def get_stopwords(self):
        stopwords_dic = open(self.stopwords_filename, encoding= 'utf-8')
        stopwords = stopwords_dic.readlines()
        stopwords = [w.strip() for w in stopwords]
        stopwords_dic.close()
        return stopwords
    
    def del_stopwords(self, word_content):
        stop_words = self.get_stopwords()
        result = []
        for sentence in word_content:
            tmp = []
            for word in sentence:
                if word not in stop_words:
                    tmp.append(word)
            result.append(tmp)
        return result
    
    def get_name_entity(self, sentence_words_content):
        result = []

        postagger = Postagger()  # 初始化实例
        postagger.load(self.pos_model_path)  # 加载模型

        recognizer = NamedEntityRecognizer()  # 初始化实例
        recognizer.load(self.ner_model_path)  # 加载模型

        for strs in sentence_words_content:
            sentence = ''.join(strs)

            # words = list(jieba.cut(sentence))  
            words = list(jieba.cut(sentence))    # 结巴分词
            postags = list(postagger.postag(words))  # 词性标注
            netags = list(recognizer.recognize(words, postags))  # 命名实体识别
            # tmp=[str(k+1)+'-'+v for k,v in enumerate(netags)]
            # print('\t'.join(tmp))
            result.append(netags)
    #         print('\t'.join(netags))
    #         print('\t'.join(words))
        postagger.release()  # 释放模型
        recognizer.release()  # 释放模型
        return result
    
    def data_process(self, text):
        test_with_punctuation = self.get_sentence_content_with_punctuation(text)

        test_no_punctuation = self.get_sentence_content(test_with_punctuation)
        test_word_with_stopwords = self.get_word_content(test_no_punctuation)
        test_word = self.del_stopwords(test_word_with_stopwords)

        test_word_name_entity = self.get_name_entity(test_word)

        return test_word, test_with_punctuation, test_word_name_entity
    
    ################################################Process Data####################################################
    
    
    #################################################Emdbedding#####################################################
    ## 频率与word_embedding
    def get_coefs(self, word, *arr): 
        return word, np.asarray(arr, dtype='float32')

    def merge_sentence_words(self, sentence_words):
        result = []
        for sentence in sentence_words:
            result += sentence
        return result

    def get_frequence(self, word_sentence_content):
        tokeners = self.merge_sentence_words(word_sentence_content)

        tokener_counter = Counter(tokeners)

        self.frequence = {w: count/len(tokeners) for w, count in tokener_counter.items()}
        ## occurences_frequences = sorted(frequence.items(), key=lambda x: x[1], reverse=True)
        pass
    
    def get_embedding_index(self):
        self.embeddings_index = dict(self.get_coefs(*o.rstrip().rsplit(' ')) for o in open(self.embeddings_index_filename, encoding='UTF-8'))
        pass
    
    ## 建立节点图
    def get_connect_graph_by_text_rank(self, tokenized_text: str, window=3):
        keywords_graph = networkx.Graph()
        tokeners = tokenized_text.split()
        for ii, t in enumerate(tokeners):
            word_tuples = [(tokeners[connect], t) 
                           for connect in range(ii-window, ii+window+1) 
                           if connect >= 0 and connect < len(tokeners)]
            keywords_graph.add_edges_from(word_tuples)

        return keywords_graph
    
    ## 获取句子向量
    def sentence_embedding(self, words):
        #weight = alpah/(alpah + p)
        #alpah is a parameter, 1e-3 ~ 1e-5
        max_fre = max(self.frequence.values())

        sentence_vec = np.zeros_like(self.embeddings_index['测试'])

        words = [w for w in words if w in self.embeddings_index]
        if words == []:
            return sentence_vec

        for w in words:
            weight = self.alpah / (self.alpah + self.frequence.get(w, max_fre))
            sentence_vec += weight * self.embeddings_index[w]

        sentence_vec /= len(words)

        return sentence_vec
    
    ################################################Emdbedding#####################################################
    
    ################################################KNN Smooth#####################################################
    
    def knn_smooth(self, unranking_sentence):
        
        window = self.knn_window
        
        sen = []
        score = []
        length = len(unranking_sentence)
        for _sen, _score in unranking_sentence.items():
            sen.append(_sen)
            score.append(_score)
        smooth_sentence = [np.mean(score[:i+window+1]) for i in range(window)] + [np.mean(score[i-window:i+window+1]) for i in range(window, length-window)] + [np.mean(score[i-window:]) for i in range(length-window, length)]

        smooth_sentence_dict = {}
        for i in range(length):
            smooth_sentence_dict[sen[i]] = smooth_sentence[i]
        print(smooth_sentence_dict)
        print('\n')
        return sorted(smooth_sentence_dict.items(), key=lambda x: x[1], reverse=True)
    
    ################################################KNN Smooth#####################################################
    
    #################################################LDA Model#####################################################
    def get_LDA(self, sentence_words_content):
        lda_load = models.ldamodel.LdaModel.load(self.lda_model_filename)
        dictionary = lda_load.id2word

        content = self.merge_sentence_words(sentence_words_content)
        content_id = dictionary.doc2bow(content)

        topic = lda_load.get_document_topics(content_id)
        sorted_topic = sorted(topic, key=lambda x:x[1], reverse=True)
        major_topic_num = sorted_topic[0][0]

        print("第1封邮件的主题分布为：\n", sorted_topic, '\n')

        str_try = lda_load.print_topic(major_topic_num, topn=self.lda_num)

        major_topic = re.findall('"[\u4e00-\u9fa5].?"', str_try)
        for i in range(len(major_topic)):
            major_topic[i] = major_topic[i][1:-1]

    #     print("第1封邮件最重要的主题为：\n", major_topic, '\n')
        return major_topic
    
    #################################################LDA Model#####################################################
    
    #################################################Key Words#####################################################
    class AttrDict(dict):
        """Dict that can get attribute by dot"""
        def __init__(self, *args, **kwargs):
            super(AttrDict, self).__init__(*args, **kwargs)
            self.__dict__ = self

    def combine(self, word_list):
        """构造在window下的单词组合，用来构造单词之间的边。

        Keyword arguments:
        word_list  --  list of str, 由单词组成的列表。
        windows    --  int, 窗口大小。
        """
        window = self.keywords_window
        
        if window < 2: window = 2
        for x in range(1, window):
            if x >= len(word_list):
                break
            word_list2 = word_list[x:]
            res = zip(word_list, word_list2)
            for r in res:
                yield r
    
    def get_word2id_id2word(self, sentence_words_content):
        id2word = corpora.Dictionary(sentence_words_content)
        word2id = {id2word[i]: i for i in range(len(id2word))}
        return word2id, id2word

    def creat_graph(self, sentence_words_content, word2id):
        graph = np.zeros((len(word2id), len(word2id)))
        for word_list in sentence_words_content:
            for w1, w2 in self.combine(word_list):
                if w1 in word2id and w2 in word2id:
                    index1 = word2id[w1]
                    index2 = word2id[w2]
                    graph[index1][index2] = 1.0
                    graph[index2][index1] = 1.0
        return graph
    
    def get_keywords(self, sentence_words_content):
        word2id, id2word = self.get_word2id_id2word(sentence_words_content)

        graph = self.creat_graph(sentence_words_content, word2id)

        sorted_words = []
        nx_graph = nx.from_numpy_matrix(graph)
        scores = nx.pagerank(nx_graph, alpha=0.85)          # this is a dict
        sorted_scores = sorted(scores.items(), key = lambda item: item[1], reverse=True)

        for index, score in sorted_scores:
    #         item = AttrDict(word=id2word[index], weight=score)
            sorted_words.append(id2word[index])

        return sorted_words
    
    #################################################Key Words#####################################################
    
    #############################################Sentence Ranking##################################################
    ## 获取句子节点排序_1：相邻句子词语共现频率
    def sentence_ranking_by_text_ranking(self, split_sentence):
        sentence_graph = self.get_connect_graph_by_text_rank(' '.join(split_sentence))
        unranking_sentence = networkx.pagerank(sentence_graph)
        return unranking_sentence
    
    ## 获取句子节点排序_2：句子向量相似度
    def get_correlations(self, sentence_words_content, sentences_content, title_words_content):
        #if isinstance(text, list): text = ' '.join(text)
        
        self.get_embedding_index()
        text_words = self.merge_sentence_words(sentence_words_content)
        all_sentence_vector = self.sentence_embedding(text_words)

        title_vector = []
        for title_sen in title_words_content:
            title_vector.append(self.sentence_embedding(title_sen))

        correlations = {}

        key_words = self.get_keywords(sentence_words_content)[:self.keywords_num]
        print(key_words)
        print('\n')

        topic_words = self.get_LDA(sentence_words_content)
        print(topic_words)
        print('\n')
        for i, sentence in enumerate(sentence_words_content):
            vec = self.sentence_embedding(sentence)

            if (vec == np.zeros_like(self.embeddings_index['测试'])).all():
                correlations[sentences_content[i]] = 0.0
                continue

            correlation = (1 - self.title_weight) * cosine(all_sentence_vector, vec) + self.title_weight * sum([cosine(ti_vec, vec) for ti_vec in title_vector])/len(sentence_words_content)
            correlations[sentences_content[i]] = correlation

            key_num = 0 
            topic_num = 0
            for word in sentence:
                if word in key_words:
                    correlations[sentences_content[i]] *= 1.1
    #                 key_num += 1 
                if word in topic_words:
                    topic_num += 1             
    #         if topic_num == 0:
    #             correlations[sentences_content[i]] *= 0.9
            if topic_num != 0:
                correlations[sentences_content[i]] *= 1.1

        correlations[sentences_content[0]] *= 2.0
        correlations[sentences_content[len(sentence_words_content) - 1]] *= 1.5

        return correlations
    #############################################Sentence Ranking##################################################
    
    ################################################## Main #######################################################   
    def get_summarization_simple(self, text, title):
        print(text)
        print('\n')
        print("文章标题：    %s"%title)
        
        score_fn = partial(self.get_correlations)
        
        sentence_words_content, sentences_content, _ = self.data_process(text)

        self.get_frequence(sentence_words_content)

        title_words_content, _, _ = self.data_process(title)

        #print(title_words_content)
        unranking_sentence = score_fn(sentence_words_content, sentences_content, title_words_content)
        selected_text = set()
        current_text = ''

        ranking_sentence = self.knn_smooth(unranking_sentence)
    #     ranking_sentence = sorted(unranking_sentence.items(), key=lambda x: x[1], reverse=True)

        for sen, _ in ranking_sentence:
            if len(current_text) < self.long_constraint:
                current_text += sen
                selected_text.add(sen)
            else:
                break

        summarized = []
        for sen in sentences_content:  # print the selected sentence by sequent
            if sen in selected_text:
                summarized.append(sen)
        return ''.join(summarized)
    
    ## 最终函数_1: TextRank
    def get_summarization_simple_with_text_rank(self, text):
        return self.get_summarization_simple(text)

    ## 最终函数_2: EmbeddingRank
    def get_summarization_simple_by_sen_embedding(self, text, title):
        return self.get_summarization_simple(text, title)
    
    ################################################## Main #######################################################

# 8. Test

In [12]:
model = Summary_Model()

In [13]:
test_1 = '7月11日，连续强降雨，让四川登上了中央气象台“头条”，涪江绵阳段水位迅速上涨，洪水一度漫过了宝成铁路涪江大桥桥墩基座，超过封锁水位。洪水在即，中国铁路成都局集团公司紧急调集两列重载货物列车，一前一后开上涪江大桥，每一列货车重量约四千吨，用“重车压梁”的方式，增强桥梁自重，抵御汹涌的洪水。从11日凌晨开始，四川境内成都、绵阳、广元等地连续强降雨，而四川北向出川大动脉—宝成铁路，便主要途径成绵广这一区域。连续的强降雨天气下，绵阳市境内的涪江水位迅速上涨，一度危及到了宝成铁路涪江大桥的安全，上午10时，水位已经超过了涪江大桥上、下行大桥的封锁水位。记者从中国铁路成都局集团公司绵阳工务段了解到，上行线涪江大桥，全长393米，建成于1953年；下行线涪江大桥，全长438米，建成于1995年。“涪江大桥上游有一个水电站，由于洪水太大，水电站已无法发挥调节水位的作用。”情况紧急，铁路部门决定采用“重车压梁”的方式，增强桥梁自重，提高洪峰对桥墩冲刷时的梁体稳定性。简单来说，就是将重量大的货物列车开上涪江大桥，用货车的自重，帮助桥梁抵御汹涌的洪水。恰好，绵阳工务段近期正在进行线路大修，铁路专用的卸砟车，正好停放在绵阳附近。迎着汹涌的洪水，两列重载货车驶向宝成铁路涪江大桥。上午10时30分，第一列46052次货车，从绵阳北站开出进入上行涪江桥。上午11时15分，第二列22001次货车，从皂角铺站进入下行涪江桥。这是两列超过45节编组的重载货物列车，业内称铁路专用卸砟车，俗称“老K车”，车厢里装载的均为铁路道砟，每辆车的砟石的重量在70吨左右。记者从绵阳工务段了解到，货车里满载的砟石、加上一列货车的自重，两列“压桥”的货运列车，每一列的重量超过四千吨。“采用重车压梁的方式来应对水害，在平时的抢险中很少用到。”据了解，在绵阳工务段，上一次采用重车压梁，至少已经是二十年前的事。下午4时许，经铁路部门观测，洪峰过后，涪江水位开始下降，目前已经低于桥梁封锁水位。从下午4点37分开始，两列火车开始撤离涪江大桥。在桥上停留约6个小时后，两列重载货物列车成功完成了“保桥任务”，宝成铁路涪江大桥平安了！'
test_title_1 = '20年来首次！四千吨级重载货车压桥抵御洪峰，宝成铁路大桥已平安。'

In [14]:
print(model.get_summarization_simple_by_sen_embedding(test_1, test_title_1))

7月11日，连续强降雨，让四川登上了中央气象台“头条”，涪江绵阳段水位迅速上涨，洪水一度漫过了宝成铁路涪江大桥桥墩基座，超过封锁水位。洪水在即，中国铁路成都局集团公司紧急调集两列重载货物列车，一前一后开上涪江大桥，每一列货车重量约四千吨，用“重车压梁”的方式，增强桥梁自重，抵御汹涌的洪水。从11日凌晨开始，四川境内成都、绵阳、广元等地连续强降雨，而四川北向出川大动脉—宝成铁路，便主要途径成绵广这一区域。连续的强降雨天气下，绵阳市境内的涪江水位迅速上涨，一度危及到了宝成铁路涪江大桥的安全，上午10时，水位已经超过了涪江大桥上、下行大桥的封锁水位。记者从中国铁路成都局集团公司绵阳工务段了解到，上行线涪江大桥，全长393米，建成于1953年；下行线涪江大桥，全长438米，建成于1995年。“涪江大桥上游有一个水电站，由于洪水太大，水电站已无法发挥调节水位的作用。”情况紧急，铁路部门决定采用“重车压梁”的方式，增强桥梁自重，提高洪峰对桥墩冲刷时的梁体稳定性。简单来说，就是将重量大的货物列车开上涪江大桥，用货车的自重，帮助桥梁抵御汹涌的洪水。恰好，绵阳工务段近期正在进行线路大修，铁路专用的卸砟车，正好停放在绵阳附近。迎着汹涌的洪水，两列重载货车驶向宝成铁路涪江大桥。上午10时30分，第一列46052次货车，从绵阳北站开出进入上行涪江桥。上午11时15分，第二列22001次货车，从皂角铺站进入下行涪江桥。这是两列超过45节编组的重载货物列车，业内称铁路专用卸砟车，俗称“老K车”，车厢里装载的均为铁路道砟，每辆车的砟石的重量在70吨左右。记者从绵阳工务段了解到，货车里满载的砟石、加上一列货车的自重，两列“压桥”的货运列车，每一列的重量超过四千吨。“采用重车压梁的方式来应对水害，在平时的抢险中很少用到。”据了解，在绵阳工务段，上一次采用重车压梁，至少已经是二十年前的事。下午4时许，经铁路部门观测，洪峰过后，涪江水位开始下降，目前已经低于桥梁封锁水位。从下午4点37分开始，两列火车开始撤离涪江大桥。在桥上停留约6个小时后，两列重载货物列车成功完成了“保桥任务”，宝成铁路涪江大桥平安了！


文章标题：    20年来首次！四千吨级重载货车压桥抵御洪峰，宝成铁路大桥已平安。
['涪江', '绵阳', '大桥', '铁路', '两列']


第1封邮件的主题分布为：
 [(12, 0.2

In [20]:
test_2 = '因和美国总统特朗普意见不一，美国情报界2号人物被踢出局！当地时间8日晚间，特朗普连发两条推特，先是解雇了现任美国国家情报副总监苏·戈登，他说：“苏·戈登是一位了不起的专家，有着漫长而杰出的职业生涯。在过去两年中，我认识了苏，并对她产生了极大的尊重。苏已经宣布她将于8月15日离职，这一天正好是丹·科茨退休的日子。不久我将任命一位新的国家情报代理总监。”紧接着，特朗普又抛出一条更具份量的推特：“很高兴向你们宣布，现国家反恐中心主任约瑟夫·马奎尔从8月15日起将担任国家情报代理总监。”按理说，特朗普作为总统，任命政府部门的主管本是“分内事”。再者，特朗普发推特换个部门主管也不是一回两回了，但让美国政界和舆论界吃惊的是，这次任命的背后大有乾坤。美国有线电视新闻网（CNN）9日称，特朗普与美国情报界积怨已久，这次仅提前一星期宣布戈登“辞职”，很可能意味着他要把本应远离政治的情报界拉进党派斗争的泥潭。美国国家情报副总监戈登在给特朗普的辞职信中写道：“因为您要求一个新的领导班子掌舵，所以我愿意于2019年8月15日辞去职务。”“我相信我们的团队拥有力量，他们不会让你和我们的国家失望。”虽说戈登字里行间似乎写得情真意切，但明眼人一看便知，戈登是在暗示：她是被逼走的。据美国媒体报道，戈登是美国情报界的元老级人物，在业内服务长达30多年，位居美国国家情报局的二把手，只是因为不顺特朗普的心，就遭到“被辞职”的命运。据美国“政治”网站8日报道，戈登曾这样评价情报机构与特朗普的关系：“我们试图让他满意，就像过去每一任总统那样，但实际上，我们这群向他报告世界正如何变化的人才是让他心情不好的原因，他对我们的工作认同与否变得一点也不重要。”同时，戈登在情报界是出了名的“无党派老黄牛”，任劳任怨，深受两党欢迎，这不得不让人担心，万一出了差错，指责可能都是冲着她“老板”来的。上个月，美国国家情报总监丹·科茨宣布即将退休。按规定，戈登应该担任代理总监，但白宫就是不允，最后选了国家反恐中心主任马奎尔担任这一职务。对于特朗普的举动，美国国会方面很不买账。众议院情报委员会主席亚当·希夫明确批评说，“这些优秀领导的去职，以及总统决心要清理任何胆敢有不同意见的人，显而易见是情报界所面临的最具挑战性的时刻”。说来也巧，美国联邦调查局前副局长安德鲁·麦凯布2018年1月眼看要退休了，却被司法部无情解雇。8日，麦凯布将司法部和联邦调查局告上法庭，指称特朗普出于“政治动机”迫使司法部解雇了他，认为这是出于“党派偏见”的“违宪行为”。不管这场官司最后结果如何，特朗普多么不能容人无疑又会在公众面前好好晒一遍。自从特朗普2017年1月入主白宫后，政府人事变动犹如“走马灯”一般。从国务卿、国防部长、司法部长、白宫办公室主任、驻联合国大使等等，换了一个又一个，绝大多数是因与特朗普不同“调”。现任国务卿蓬佩奥在一次聚会上也无奈地说，说不定哪一天，他就在总统的推特上被解雇了。有分析认为，特朗普就是一根筋，眼里容不得一粒沙子。其上任以来，先是大骂媒体制造假新闻，只有他的推特才是真新闻。对于朝鲜半岛核问题、“伊斯兰国”的威胁、阿富汗局势以及伊朗核问题等等，情报官员给出的中肯意见，特朗普都不屑一顾。如今他对情报界开刀，顺之则留，逆之则去，将是这位总统的“不二法则”。'
test_title_2 = '美情报界二号人物“被辞职”，美媒猜原因：总让特朗普“心情不好”'

In [21]:
#get_summarization_simple_by_sen_embedding(test_1, test_title_1, frequence, embeddings_index, constraint=200)
print(model.get_summarization_simple_by_sen_embedding(test_2, test_title_2))

因和美国总统特朗普意见不一，美国情报界2号人物被踢出局！当地时间8日晚间，特朗普连发两条推特，先是解雇了现任美国国家情报副总监苏·戈登，他说：“苏·戈登是一位了不起的专家，有着漫长而杰出的职业生涯。在过去两年中，我认识了苏，并对她产生了极大的尊重。苏已经宣布她将于8月15日离职，这一天正好是丹·科茨退休的日子。不久我将任命一位新的国家情报代理总监。”紧接着，特朗普又抛出一条更具份量的推特：“很高兴向你们宣布，现国家反恐中心主任约瑟夫·马奎尔从8月15日起将担任国家情报代理总监。”按理说，特朗普作为总统，任命政府部门的主管本是“分内事”。再者，特朗普发推特换个部门主管也不是一回两回了，但让美国政界和舆论界吃惊的是，这次任命的背后大有乾坤。美国有线电视新闻网（CNN）9日称，特朗普与美国情报界积怨已久，这次仅提前一星期宣布戈登“辞职”，很可能意味着他要把本应远离政治的情报界拉进党派斗争的泥潭。美国国家情报副总监戈登在给特朗普的辞职信中写道：“因为您要求一个新的领导班子掌舵，所以我愿意于2019年8月15日辞去职务。”“我相信我们的团队拥有力量，他们不会让你和我们的国家失望。”虽说戈登字里行间似乎写得情真意切，但明眼人一看便知，戈登是在暗示：她是被逼走的。据美国媒体报道，戈登是美国情报界的元老级人物，在业内服务长达30多年，位居美国国家情报局的二把手，只是因为不顺特朗普的心，就遭到“被辞职”的命运。据美国“政治”网站8日报道，戈登曾这样评价情报机构与特朗普的关系：“我们试图让他满意，就像过去每一任总统那样，但实际上，我们这群向他报告世界正如何变化的人才是让他心情不好的原因，他对我们的工作认同与否变得一点也不重要。”同时，戈登在情报界是出了名的“无党派老黄牛”，任劳任怨，深受两党欢迎，这不得不让人担心，万一出了差错，指责可能都是冲着她“老板”来的。上个月，美国国家情报总监丹·科茨宣布即将退休。按规定，戈登应该担任代理总监，但白宫就是不允，最后选了国家反恐中心主任马奎尔担任这一职务。对于特朗普的举动，美国国会方面很不买账。众议院情报委员会主席亚当·希夫明确批评说，“这些优秀领导的去职，以及总统决心要清理任何胆敢有不同意见的人，显而易见是情报界所面临的最具挑战性的时刻”。说来也巧，美国联邦调查局前副局长安德鲁·麦凯布2018年1月眼看要退休了，却被司法部无情解雇。8日，麦凯布将

In [22]:
test_3 = '参考消息网8月15日报道 英国《卫报》网站8月11日发表题为《为何特朗普的“极限施压”外交政策收效甚微？》的文章，作者为朱利安·博格。文章称，唐纳德·特朗普即将带着没有明显外交胜利的政绩投入2020年大选。一些戏剧性的失败和世界各地一系列迫在眉睫的危机可能会破坏他争取连任的努力。因此，许多人预测，特朗普将寻求通过在全球采取影响深远但结果难料的干预行动来扭转这一趋势——这将使未来16个月的局势比他担任总统以来更为动荡。文章称，外交政策问题很少在美国总统大选中发挥主导作用，但特朗普却不遗余力地将自己塑造为能与外国领导人达成协议、并将美军撤回国内的唯一人选。一些外国危机对美国国内产生了直接影响。特朗普承诺的中美协议至今没有达成，加征关税和贸易战愈演愈烈，这些都将直接影响到消费者和生产商。奥巴马的演讲撰稿人、外交政策顾问本·罗兹说：“有意思的是，在这其中的每一个问题上，特朗普都看到了在国内政治中可以带来的好处：对中国、伊朗和委内瑞拉强硬，心中想的是争取佛罗里达选民……但无论短期利益如何，都已被他制造的长期混乱蒙上了阴影。”文章认为，即将全面开启竞选模式的特朗普似乎意识到了自己在外交政策上的不足，并寻求解决这个问题。从短期来看，这意味着重新定义成功。文章称，在朝鲜问题上，这意味着淡化他最初的说法，即与朝鲜的外交最终将带来核裁军。一直以来，他强调的是朝鲜暂停核试验和导弹试验。当朝鲜加紧发射短程导弹时，红线就被移至防止其发射洲际导弹上。文章还称，针对伊朗的“极限施压”战略旨在改变伊朗政府在中东地区的行为方式，结束其铀浓缩活动，使其放弃直接或通过代理人介入叙利亚、伊拉克和也门事务，并说服伊朗与美国开启新的谈判。以这些标准来衡量，到目前为止，这项战略正在产生事与愿违的效果。伊朗正在突破2015年核协议的限制，加紧推进核计划。目前尚不清楚，随着朝鲜越来越多地进行的导弹发射，以及伊朗突破核限制，降低成功门槛的战术能否使特朗普在明年11月的大选中获胜。文章称，特朗普的另一个选择是，采取大胆的行动重新掌握主动权。这将可能是军事形式，但也是最后的手段。无论对选举还是对其他方面而言，朝鲜半岛或波斯湾再发生冲突将是灾难性的。今年6月，特朗普打击伊朗的行动“已经箭在弦上”，却在最后一刻取消了行动。当时有人警告他说，这将可能使他失去第二个任期。文章还称，他本能地想要做一笔引人注目的交易。他表示有意与朝鲜领导人再次举行峰会。在峰会上，他可以提出以拆除一些朝鲜核武器基础设施为条件，取消部分制裁措施。迄今为止，特朗普寻求与伊朗领导人直接对话的努力都遭到拒绝，但9月的联合国大会提供了另一个与伊朗总统哈桑·鲁哈尼会晤的机会。要做到这一点，就可能需要极大缓解对伊朗的经济施压。特朗普也许认为值得冒这个险，即使这需要他的极端鹰派国家安全顾问约翰·博尔顿下台。文章指出，中美也可能达成部分贸易协议。特朗普将可能利用任何这样的协议“大做文章”，称其为世纪性的协议。曾在奥巴马政府担任高级国家安全官员的朱莉安娜·史密斯说：“特朗普将抢占头条，将自己塑造成领袖。”她说：“未来18个月，我们将看到一系列轻率和鲁莽的举动以及更多的戏剧性事件。所以，坐稳了！”'
test_title_3 = '英媒预测特朗普或制造“大交易”提升连任几率'

In [23]:
print(model.get_summarization_simple_by_sen_embedding(test_3, test_title_3))

参考消息网8月15日报道 英国《卫报》网站8月11日发表题为《为何特朗普的“极限施压”外交政策收效甚微？》的文章，作者为朱利安·博格。文章称，唐纳德·特朗普即将带着没有明显外交胜利的政绩投入2020年大选。一些戏剧性的失败和世界各地一系列迫在眉睫的危机可能会破坏他争取连任的努力。因此，许多人预测，特朗普将寻求通过在全球采取影响深远但结果难料的干预行动来扭转这一趋势——这将使未来16个月的局势比他担任总统以来更为动荡。文章称，外交政策问题很少在美国总统大选中发挥主导作用，但特朗普却不遗余力地将自己塑造为能与外国领导人达成协议、并将美军撤回国内的唯一人选。一些外国危机对美国国内产生了直接影响。特朗普承诺的中美协议至今没有达成，加征关税和贸易战愈演愈烈，这些都将直接影响到消费者和生产商。奥巴马的演讲撰稿人、外交政策顾问本·罗兹说：“有意思的是，在这其中的每一个问题上，特朗普都看到了在国内政治中可以带来的好处：对中国、伊朗和委内瑞拉强硬，心中想的是争取佛罗里达选民……但无论短期利益如何，都已被他制造的长期混乱蒙上了阴影。”文章认为，即将全面开启竞选模式的特朗普似乎意识到了自己在外交政策上的不足，并寻求解决这个问题。从短期来看，这意味着重新定义成功。文章称，在朝鲜问题上，这意味着淡化他最初的说法，即与朝鲜的外交最终将带来核裁军。一直以来，他强调的是朝鲜暂停核试验和导弹试验。当朝鲜加紧发射短程导弹时，红线就被移至防止其发射洲际导弹上。文章还称，针对伊朗的“极限施压”战略旨在改变伊朗政府在中东地区的行为方式，结束其铀浓缩活动，使其放弃直接或通过代理人介入叙利亚、伊拉克和也门事务，并说服伊朗与美国开启新的谈判。以这些标准来衡量，到目前为止，这项战略正在产生事与愿违的效果。伊朗正在突破2015年核协议的限制，加紧推进核计划。目前尚不清楚，随着朝鲜越来越多地进行的导弹发射，以及伊朗突破核限制，降低成功门槛的战术能否使特朗普在明年11月的大选中获胜。文章称，特朗普的另一个选择是，采取大胆的行动重新掌握主动权。这将可能是军事形式，但也是最后的手段。无论对选举还是对其他方面而言，朝鲜半岛或波斯湾再发生冲突将是灾难性的。今年6月，特朗普打击伊朗的行动“已经箭在弦上”，却在最后一刻取消了行动。当时有人警告他说，这将可能使他失去第二个任期。文章还称，他本能地想要做一笔引人注目的交易。他表示有意与朝